<h3><center>World Inequality</center></h3>
<h5><center><a href="https://github.com/Aarsh-Wankar/">Aarsh Wankar</a>, IIT Gandhinagar, <a style="text-decoration:none" href="mailto:23110003@iitgn.ac.in">23110003@iitgn.ac.in
<br><br>
<a style="text-decoration:none" href="https://github.com/jsmaskeen/">Jaskirat Singh Maskeen</a>, IIT Gandhinagar, <a style="text-decoration:none" href="mailto:23110146@iitgn.ac.in">23110146@iitgn.ac.in</a>
<br><br>
<a style="text-decoration:none" href="https://github.com/nishchaybhutoria/">Nishchay Bhutoria</a>, IIT Gandhinagar, <a style="text-decoration:none" href="mailto:23110222@iitgn.ac.in">23110222@iitgn.ac.in</a>
<br><br>
<a style="text-decoration:none" href="https://github.com/ridhamp4/">Ridham Patel</a>, IIT Gandhinagar, <a style="text-decoration:none" href="mailto:23110238@iitgn.ac.in">23110238@iitgn.ac.in</a>

</h5>

<center><a href="https://github.com/jsmaskeen/CS328-Writing-Assignment/"><img src="https://img.shields.io/badge/Github-Writing%20Assignment-brightgreen" alt="Repo"></a></center>

In [1]:
from IPython.display import HTML
import plotly.io as pio
pio.renderers.default = "notebook_connected"
HTML('''<button type="button" class="btn btn-outline-danger"  onclick="codeToggle();">Toggle Code</button>''')


#### Imports

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import tarfile
import io


## Exploring the data


#### Loading and cleaning the data

In [ ]:
# takes 1-3 mins
country_dataframes = {}
metadata_dataframes = {}
country_code_region_df = None
with tarfile.open(r'data\wid_all_data.tar.gz', 'r:gz') as tar:
    for member in tar.getmembers():
        if member.isfile():
            file_obj = tar.extractfile(member)
            if file_obj is None:
                continue
            filename = member.name
            if filename.endswith('.csv'):
                df = pd.read_csv(file_obj,delimiter=';')
                if 'WID_countries' in filename:
                    country_code_region_df = df
                elif 'metadata' in filename:
                    metadata_dataframes[filename.split('/')[1][:-4]] = df
                else:
                    country_dataframes[filename.split('/')[1][:-4]] = df

In [24]:
continent_codes = {
    "Africa": "QB",
    "Asia": "QD",
    "Europe": "QE",
    "North America": "QP",
    "South America": "QR",
    "Oceania": "QF"
}
continent_df = {key:country_dataframes[f'WID_data_{val}'] for key,val in continent_codes.items()}


In [29]:
deleted_rows = country_code_region_df[country_code_region_df.isnull().any(axis=1)]
deleted_rows

,alpha2,titlename,shortname,region,region2
45,CN-RU,rural China,Rural China,NaN,NaN
46,CN-UR,urban China,Urban China,NaN,NaN
57,DE-BD,Baden,Baden,NaN,NaN
58,DE-BY,Bavaria,Bavaria,NaN,NaN
59,DE-HB,Bremen,Bremen,NaN,NaN
...,...,...,...,...,...
389,XQ-MER,NaN,NaN,NaN,NaN
390,XR,Russia & Central Asia,Russia & Central Asia,NaN,NaN
391,XR-MER,Russia & Central Asia (at market exchange rate),Russia & Central Asia (at market exchange rate),NaN,NaN
392,XS,South & South-East Asia,South & South-East Asia,NaN,NaN


In [21]:
print(list(zip(deleted_rows['alpha2'].to_list(),deleted_rows['titlename'].to_list())))
print()

[('CN-RU', 'rural China'), ('CN-UR', 'urban China'), ('DE-BD', 'Baden'), ('DE-BY', 'Bavaria'), ('DE-HB', 'Bremen'), ('DE-HE', 'Hesse'), ('DE-HH', 'Hamburg'), ('DE-PR', 'Prussia'), ('DE-SN', 'Saxony'), ('DE-WU', 'Wurttemberg'), (nan, 'Namibia'), ('OA', 'Other Russia & Central Asia'), ('OA-MER', 'Other Russia & Central Asia (at market exchange rate)'), ('OB', 'Other East Asia'), ('OB-MER', 'Other East Asia (at market exchange rate)'), ('OC', 'Other Western Europe'), ('OC-MER', 'Other Western Europe (at market exchange rate)'), ('OD', 'Other Latin America'), ('OD-MER', 'Other Latin America (at market exchange rate)'), ('OE', 'Other MENA'), ('OE-MER', 'Other MENA (at market exchange rate)'), ('OH', 'Other North America & Oceania'), ('OH-MER', 'Other North America & Oceania (at market exchange rate)'), ('OI', 'Other South & South-East Asia'), ('OI-MER', 'Other South & South-East Asia (at market exchange rate)'), ('OJ', 'Other Sub-Saharan Africa'), ('OJ-MER', 'Other Sub-Saharan Africa (at ma

In [27]:
metadata_dataframes['WID_metadata_IN']

,country,variable,age,pop,countryname,shortname,simpledes,technicaldes,shorttype,longtype,shortpop,longpop,shortage,longage,unit,source,method,extrapolation,data_points
0,IN,acfccoi992,992,i,India,Consumption of fixed capital of corporations,NaN,NaN,Average,Average income or wealth between two percentil...,individuals,The base unit is the individual (rather than t...,Adults,The population is comprised of individuals ove...,INR,See [URL][URL_TEXT]DINA guidelines[/URL_TEXT][...,WID.world estimations as a proportion of GDP b...,NaN,NaN
1,IN,acfccoi999,999,i,India,Consumption of fixed capital of corporations,NaN,NaN,Average,Average income or wealth between two percentil...,individuals,The base unit is the individual (rather than t...,All Ages,The population is comprised of individuals of ...,INR,See [URL][URL_TEXT]DINA guidelines[/URL_TEXT][...,WID.world estimations as a proportion of GDP b...,NaN,NaN
2,IN,acfcfci992,992,i,India,Consumption of fixed capital of financial copo...,NaN,NaN,Average,Average income or wealth between two percentil...,individuals,The base unit is the individual (rather than t...,Adults,The population is comprised of individuals ove...,INR,See [URL][URL_TEXT]DINA guidelines[/URL_TEXT][...,WID.world estimations as a proportion of GDP b...,NaN,NaN
3,IN,acfcfci999,999,i,India,Consumption of fixed capital of financial copo...,NaN,NaN,Average,Average income or wealth between two percentil...,individuals,The base unit is the individual (rather than t...,All Ages,The population is comprised of individuals of ...,INR,See [URL][URL_TEXT]DINA guidelines[/URL_TEXT][...,WID.world estimations as a proportion of GDP b...,NaN,NaN
4,IN,acfcgoi992,992,i,India,Consumption of fixed capital of the general go...,NaN,NaN,Average,Average income or wealth between two percentil...,individuals,The base unit is the individual (rather than t...,Adults,The population is comprised of individuals ove...,INR,See [URL][URL_TEXT]DINA guidelines[/URL_TEXT][...,WID.world estimations as a proportion of GDP b...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1101,IN,xlceuxi999,999,i,India,"Market exchange rate, LCU per EUR",Official exchange rate of the local currency t...,NaN,Exchange rates,Exchange rate series,individuals,The base unit is the individual (rather than t...,All Ages,The population is comprised of individuals of ...,local currency per foreign currency,NaN,NaN,NaN,NaN
1102,IN,xlcuspi999,999,i,India,"PPP conversion factor, LCU per USD",PPP conversion factor of current local currenc...,NaN,Exchange rates,Exchange rate series,individuals,The base unit is the individual (rather than t...,All Ages,The population is comprised of individuals of ...,local currency per foreign currency,[URL][URL_LINK]http://stats.oecd.org/Index.asp...,We extrapolate the PPP from the latest ICP (20...,NaN,NaN
1103,IN,xlcusxi999,999,i,India,"Market exchange rate, LCU per USD",Official exchange rate of the local currency t...,NaN,Exchange rates,Exchange rate series,individuals,The base unit is the individual (rather than t...,All Ages,The population is comprised of individuals of ...,local currency per foreign currency,NaN,NaN,NaN,NaN
1104,IN,xlcyupi999,999,i,India,"PPP conversion factor, LCU per CNY",PPP conversion factor of current local currenc...,NaN,Exchange rates,Exchange rate series,individuals,The base unit is the individual (rather than t...,All Ages,The population is comprised of individuals of ...,local currency per foreign currency,[URL][URL_LINK]http://stats.oecd.org/Index.asp...,We extrapolate the PPP from the latest ICP (20...,NaN,NaN


In [28]:
country_dataframes['WID_data_IN'][country_dataframes['WID_data_IN']['variable'] == 'acfccoi992']

,country,variable,percentile,year,value,age,pop
1495,IN,acfccoi992,p0p100,1951,99.5,992,i
1496,IN,acfccoi992,p0p100,1952,128.7,992,i
1497,IN,acfccoi992,p0p100,1953,162.6,992,i
1498,IN,acfccoi992,p0p100,1954,161.4,992,i
1499,IN,acfccoi992,p0p100,1955,206.2,992,i
...,...,...,...,...,...,...,...
1559,IN,acfccoi992,p0p100,2015,11607.5,992,i
1560,IN,acfccoi992,p0p100,2016,12163.4,992,i
1561,IN,acfccoi992,p0p100,2017,12553.6,992,i
1562,IN,acfccoi992,p0p100,2018,13424.4,992,i


In [36]:
continent_df['North America']

,country,variable,percentile,year,value,age,pop
0,QP,ehfcari999,p0p100,1990,2466.781094,999,i
1,QP,ehfcari999,p0p100,1991,2460.412382,999,i
2,QP,ehfcari999,p0p100,1992,2562.497751,999,i
3,QP,ehfcari999,p0p100,1993,2551.664727,999,i
4,QP,ehfcari999,p0p100,1994,2574.631642,999,i
...,...,...,...,...,...,...,...
195225,QP,tptincj992,p9p100,2019,4297.100000,992,j
195226,QP,tptincj992,p9p100,2020,4206.200000,992,j
195227,QP,tptincj992,p9p100,2021,4955.400000,992,j
195228,QP,tptincj992,p9p100,2022,5065.900000,992,j


In [42]:
metadata_dataframes['WID_metadata_QP'].to_clipboard()

In [ ]:
continent_df['North America'][(continent_df['North America']['percentile'] == 'p99.9p100') & (continent_df['North America']['variable'] == 'acomhni99')]

,country,variable,percentile,year,value,age,pop


In [ ]:
def remove_null_values(dataframes):
    ...

In [46]:
df = dataframes['wid_all_data/WID_countries.csv']
empty_rows = df[df.isnull().any(axis=1)]
empty_rows

,alpha2,titlename,shortname,region,region2


In [12]:
fig = px.scatter_3d(iris_df, x='sepal_length', y='sepal_width', z='petal_width',
              color='species', template="plotly_dark")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

# Jaskirat

# Aarsh

# Ridham

# Nishchay